In [1]:
import json
import pandas as pd


stock_file = open('D:\\Python\\python-works\\grouping-minimze-cost-waste\\stocks.json')
stock = json.load(stock_file)

data = open('D:\\Python\\python-works\\grouping-minimze-cost-waste\\openai_response.json')
data = json.load(data)
df = pd.json_normalize(data)

In [2]:
from functools import cmp_to_key
import pdfplumber
import io
import traceback
import openpyxl
import json
from jinja2 import Environment, FileSystemLoader
import pandas as pd
from datetime import datetime
from fastapi import UploadFile, HTTPException
from openai import OpenAI
from io import StringIO
from dotenv import load_dotenv
import os
from collections import defaultdict
from openpyxl import load_workbook, Workbook, styles

In [45]:
import pdfkit
from bidi import algorithm as bidi_algorithm
template_dir = os.path.join(os.path.dirname("D:\\Python\\python-works\\grouping-minimze-cost-waste\\"), 'templates')
print(template_dir)
env = Environment(loader=FileSystemLoader(template_dir))

import re

def is_hebrew(text):
    # Check if the text contains Hebrew characters
    hebrew_pattern = re.compile(r'[\u0590-\u05FF]')
    return bool(hebrew_pattern.search(text))

def save_results_to_dataframe(results, bars, projectName):
    now = datetime.now()
    today_date = now.date()

    # Overview Data
    total_cuts = len(results)
    total_profiles = len(set([bar['profile'] for bar in bars]))
    total_length_m = sum([bar['stock_length']
                         for bar in bars]) / 1000  # Convert mm to m
    total_waste_m = sum([bar['remaining_waste']
                        for bar in bars]) / 1000  # Convert mm to m
    
    overview_template = env.get_template('overview_template.html.j2')
    print(is_hebrew(projectName))
    overview_template_html = overview_template.render({
        "projectName": "" if is_hebrew(projectName) else projectName,
        "dateGenerated":today_date.strftime("%d/%m/%Y"),
        "totalProfiles":total_profiles,
        "totalLengthM": total_length_m,
        "totalWasteM":total_waste_m,
        "totalCuts":total_cuts,
        "isHebrew": is_hebrew(projectName)
    })

    overview_data = [
        ["Overview and Summary", ""],
        ["Project Name", projectName],
        ["Date Generated", today_date],
        ["Total Profiles Type", total_profiles],
        ["Total Length (m)", total_length_m],
        ["Total Waste (m)", total_waste_m],
        ["Total Cuts", total_cuts],
    ]
    overview_df = pd.DataFrame(overview_data, columns=["", ""])

    material_agg = defaultdict(
        lambda: {'quantity': 0, 'total_length_m': 0.0, 'total_waste_mm': 0})

    for bar in bars:
        key = (bar['profile'], bar['stock_length'])
        material_agg[key]['quantity'] += 1
        # Convert mm to m
        material_agg[key]['total_length_m'] += bar['stock_length'] / 1000.0
        material_agg[key]['total_waste_mm'] += bar['remaining_waste']


    # Material Data
    material_data = []
    for (profile, stock_length), data in material_agg.items():
        # Calculate estimated waste percentage
        total_stock_length_mm = stock_length * data['quantity']
        waste_percentage = (
            data['total_waste_mm'] / total_stock_length_mm) * 100 if total_stock_length_mm else 0
        material_data.append([
            profile,
            stock_length,
            data['quantity'],
            round(data['total_length_m'], 2),
            round(waste_percentage, 2)
        ])

    cutlist_template = env.get_template('cutlist_template.html.j2')
    cutlist_template_html = cutlist_template.render(data = material_data)

    material_df = pd.DataFrame(material_data, columns=[
        "Profile Type", "Stock Length (mm)", "Quantity",
        "Total Length (m)", "Est Waste (%)"
    ])

    stock_profile_template_data = []

    # Cut List Data
    cut_data = []
    for bar in bars:
        entity = {
            "stockProfilePOS": bar['id'],
            "profileType": bar['profile'],
            "stockLength": bar['stock_length'],
            "wasteMM": bar['remaining_waste'],
            "cuts": []
        }
        # Empty row for separation
        cut_data.append(["Stock profile POS", bar['id']])
        # Empty row for separation
        cut_data.append(["Profile Type", bar['profile']])
        # Empty row for separation
        cut_data.append(["Stock Length", bar['stock_length']])
        cut_data.append(["Waste (mm)", bar['remaining_waste']])

        cut_data.append(["Pos Number", "Cut Lengths (mm)", "Quantity"])


        # Group cuts by 'posNumber' and 'cut_length'
        cut_counts = defaultdict(int)
        for cut in bar['cuts']:
            key = (cut['posNumber'], cut['cut_length'])
            cut_counts[key] += 1

        # Convert the grouped cuts into a list and sort for consistency
        grouped_cuts = [(posNumber, cut_length, quantity)
                        for (posNumber, cut_length), quantity in cut_counts.items()]
        # Sort by Pos Number and Cut Length
        grouped_cuts.sort(key=lambda x: (x[0], x[1]))

        # Add grouped cuts to cut_data
        for posNumber, cut_length, quantity in grouped_cuts:
            entity["cuts"].append({
                "posNumber": posNumber,
                "cutLength": cut_length,
                "quantity": quantity
            })
            cut_data.append([posNumber, cut_length, quantity])
        
        stock_profile_template_data.append(entity)
        cut_data.append([])  # Empty row for separation

    stock_profile_template = env.get_template('stock_profile_template.html.j2')
    stock_profile_html = stock_profile_template.render(data=stock_profile_template_data)

    cut_list_df = pd.DataFrame(cut_data)

    # Write to Excel
    try:
        with pd.ExcelWriter("Combined_Output.xlsx", engine="openpyxl") as writer:
            overview_df.to_excel(
                writer, sheet_name="Overview", index=False, header=False)
            material_df.to_excel(
                writer, sheet_name="Material List", index=False)
            cut_list_df.to_excel(
                writer, sheet_name="Cut List", index=False, header=False)

        # Merge Sheets
        pdf = merge_sheets_into_one("Combined_Output.xlsx", [
                              "Overview", "Material List", "Cut List"], overview_template_html, cutlist_template_html, projectName ,stock_profile_html)
        return pdf
    except Exception as e:
        traceback.print_exc()


def merge_sheets_into_one(file_path, sheet_names, overview_html, cutlist_html, stock_profile_html ,projectName, output_sheet_name="MergedSheet"):
    # Load the workbook and select sheets to merge
    original_wb = load_workbook(file_path)
    new_wb = Workbook()
    merged_sheet = new_wb.active
    merged_sheet.title = "Data"  # Name for the merged sheet in the new workbook

    # Row counter to keep track of where to paste data in the merged sheet
    current_row = 1
    first_blank_row_encountered = False
    # Flag to ensure we start writing data only after valid rows begin
    has_written_data = False
    first_row_written = False  # Flag to indicate if the first row has been written
    # Border style
    bottom_border = styles.Border(bottom=styles.Side(
        border_style='thin', color='000000'))

    # Loop through each sheet specified in sheet_names
    for sheet_name in sheet_names:
        sheet = original_wb[sheet_name]
        if sheet_name == "Material List":
            merged_sheet.cell(row=current_row, column=1,
                              value="Material and Cut List")
            current_row += 2  # Move to the next row after adding the bold text

        elif sheet_name == "Cut List":
            current_row += 1
            merged_sheet.cell(row=current_row, column=1, value="Cut List and Optimization").font = styles.Font(
                bold=True, size=12)
            current_row += 2

        # Copy each row from the current sheet to the merged sheet
        for row in sheet.iter_rows(values_only=True):
            if not has_written_data and all(cell is None for cell in row):
                continue  # Skip to the next row
            has_written_data = True

            # Check if the current row is blank
            if all(cell is None for cell in row):
                # Leave the first encountered blank row empty and continue
                if not first_blank_row_encountered:
                    first_blank_row_encountered = True
                    current_row += 1  # Move to the next row in the merged sheet for insertion
                    continue

            # Copy each row from the current sheet to the merged sheet
            for col_num, cell_value in enumerate(row, start=1):
                cell = merged_sheet.cell(
                    row=current_row, column=col_num, value=cell_value)
            # Check if the cell in column A (first cell in the row) contains 'Waste'
            cell_in_column_a = row[0] if len(row) > 0 else None
            if cell_in_column_a and 'Waste' in str(cell_in_column_a):
                current_row += 1  # Insert a blank row by incrementing current_row
            current_row += 1  # Move to the next row in the merged sheet
            if not first_row_written:
                first_row_written = True
                current_row += 1

        current_row += 1

    # Apply bold formatting to the first column in the merged sheet
    for row in merged_sheet.iter_rows(min_row=1, max_row=current_row - 1, min_col=1, max_col=1):
        for cell in row:
            cell.font = styles.Font(bold=True)

    # Apply bold formatting to row 9 in the merged sheet
    for cell in merged_sheet[12]:  # Access all cells in row 9
        cell.font = styles.Font(bold=True)

    for row in range(1, current_row):  # Start from row 9 to the last row
        cell = merged_sheet.cell(row=row, column=2)  # Column 2 is "B"
        cell.alignment = styles.Alignment(horizontal="left")
    merged_sheet['A1'].font = styles.Font(bold=True, size=12)
    merged_sheet['A10'].font = styles.Font(bold=True, size=12)

    # Apply border and reset font starting from row 12 until the next blank row after row 12
    for row in range(13, current_row):
        # Check if the row is empty, if it is, stop applying styles
        if all(cell.value is None for cell in merged_sheet[row]):
            current_row += 1  # Move to the next row after the empty row
            break  # Stop processing further rows if a blank row is encountered

        for col_num in range(1, merged_sheet.max_column + 1):
            cell = merged_sheet.cell(row=row, column=col_num)
            # Apply the bottom border style to the cell
            cell.border = bottom_border
            # Reset the font to simple (non-bold)
            cell.font = styles.Font(bold=False)
    row_cell = merged_sheet.cell(row=1, column=1)
    # Iterate over cells in column A and make "Cut List and Optimization" bold with size 12
    row_tracker = 0
    for row in merged_sheet.iter_rows(min_row=1, max_row=current_row, min_col=1, max_col=1):
        for cell in row:
            if cell.value == "Cut List and Optimization":
                cell.font = styles.Font(bold=True, size=12)
                current_row += 1
                row_tracker = cell.row

            if cell.value == "Pos Number":
                for col_num in range(1, merged_sheet.max_column + 1):
                    row_cell = merged_sheet.cell(row=cell.row, column=col_num)
                    row_cell.font = styles.Font(bold=True)
            # Reset the font to simple (non-bold) after "Pos Number" row until the next "Pos Number" is detected
            if cell.value == "Pos Number":
                pos_number_row = cell.row
                for row in range(pos_number_row + 1, current_row):
                    next_cell = merged_sheet.cell(row=row, column=1)
                    if next_cell.value == "Stock profile POS":
                        break
                    for col_num in range(1, merged_sheet.max_column + 1):
                        row_cell = merged_sheet.cell(row=row, column=col_num)
                        row_cell.font = styles.Font(bold=False)

    # Apply bottom border to all cells with values after row_tracker
    for row in range(row_tracker + 1, current_row):
        for col_num in range(1, merged_sheet.max_column + 1):
            cell = merged_sheet.cell(row=row, column=col_num)
            if cell.value is not None:
                cell.border = bottom_border

    # Save the workbook (you can choose to overwrite or save as a new file)
    pdf = save_to_pdf(overview_html, cutlist_html, stock_profile_html, projectName)
    return pdf

from xhtml2pdf import pisa
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
from io import BytesIO
import io

def save_to_pdf(overview_html, cutlist_html, stock_profile_html, projectName):
    # Step 1: Generate the initial PDF using xhtml2pdf
    html_content = f"<!DOCTYPE html><html><head><meta charset=\"UTF-8\"></head><body>{overview_html+cutlist_html+stock_profile_html}</body></html>"
    pdf_buffer = io.BytesIO()
    pisa_status = pisa.CreatePDF(html_content.encode(encoding="utf-8"), dest=pdf_buffer, encoding='utf-8')

    # Check for errors in PDF generation
    if pisa_status.err:
        print("Error in PDF creation:", pisa_status.err)
        return None

    pdf_buffer.seek(0)  # Rewind the buffer to the beginning for reading

    # Step 2: Add Hebrew text to the first page
    modified_pdf_buffer = BytesIO()
    reader = PdfReader(pdf_buffer)
    writer = PdfWriter()

    # Register a font that supports Hebrew (e.g., Arial)
    pdfmetrics.registerFont(TTFont('Noto Sans Hebrew', 'templates\\font\\Calibri.ttf'))  # Ensure arial.ttf is available on your system

    # Create overlay with Hebrew text
    overlay_buffer = BytesIO()
    can = canvas.Canvas(overlay_buffer)
    can.setFont("Noto Sans Hebrew", 8)  # Use the registered Hebrew-supported font
    can.drawString(100, 700, "שלום עולם")  # Adjust position as needed
    can.save()
    overlay_buffer.seek(0)

    # Read the overlay PDF
    overlay_reader = PdfReader(overlay_buffer)

    # Merge the overlay onto the first page
    first_page = reader.pages[0]
    first_page.merge_page(overlay_reader.pages[0])
    writer.add_page(first_page)

    # Add the remaining pages
    for page in reader.pages[1:]:
        writer.add_page(page)

    # Write the modified PDF to the new buffer
    writer.write(modified_pdf_buffer)
    modified_pdf_buffer.seek(0)  # Reset buffer position

    return modified_pdf_buffer


D:\Python\python-works\grouping-minimze-cost-waste\templates


In [46]:
from itertools import groupby

def calculate_cuts(df, stock_lengths):
    """Calculate cuts based on stock availability, track waste per profile, and track bars used."""
    # Convert the stock lengths to mm (from meters)
    stock_lengths = {key: [float(length) * 1000 for length in lengths]
                     for key, lengths in stock_lengths.items()}

    results = []
    cuts = []
    bars = []

    # Generate the cuts list (profile, length, posNumber) for each row in the dataframe
    for _, row in df.iterrows():
        posNumber = row['posNumber']
        profile = row['profile']
        quantity = int(row['quantity'])
        length = row['length']
        if isinstance(length, str):
            length = float(length.split(" ")[0])
        else:
            length = float(length)
        
        # Add the cuts for each profile
        cuts.extend([(profile, length, posNumber) for _ in range(quantity)])

    # Group the cuts by profile type
    grouped_cuts = {}
    for profile, cut_group in groupby(sorted(cuts, key=lambda x: x[0]), key=lambda x: x[0]):
        grouped_cuts[profile] = list(cut_group)

    # Now we'll pack cuts for each profile into the available bars
    packed_bars = {}

    for profile, cuts_list in grouped_cuts.items():
        available_bars = stock_lengths.get(profile, [])
        cuts_list_sorted = sorted(cuts_list, key=lambda x: x[1], reverse=True)  # Sort cuts by length, descending
        packed_bars[profile] = []

        # Sort the bars in descending order (prefer larger bars first)
        available_bars_sorted = sorted(available_bars, reverse=True)

        remaining_cuts = cuts_list_sorted[:]

        # Try to fit cuts into bars with minimum waste
        while remaining_cuts:
            for bar_length in available_bars_sorted:
                bar = []
                bar_used = 0  # Keeps track of the current bar usage

                # Try to fit cuts into the current bar
                for cut in remaining_cuts[:]:  # Iterating over remaining cuts
                    cut_length = cut[1]  # Cut length is the second element in the tuple
                    if cut_length <= bar_length - bar_used:  # Check if the cut fits in the current bar
                        bar.append(cut)  # Add the cut to the current bar
                        bar_used += cut_length
                        remaining_cuts.remove(cut)  # Remove the cut from remaining cuts

                # If anything was packed into the bar, add the bar to the packed_bars
                if bar:
                    cuts_sum = sum([x[1] for x in bar])
                    bar_should_used = -1
                    for b in available_bars_sorted:
                        if b >= cuts_sum:
                            bar_should_used = b
                    packed_bars[profile].append({
                        'bar_length': bar_should_used,
                        'cuts': bar
                    })

                # If all remaining cuts are packed, stop
                if not remaining_cuts:
                    break
                break

    json_file_path = os.path.join(os.path.dirname("D:\\Python\\python-works\\grouping-minimze-cost-waste\\"), 'grouped_cuts.json')
    # Save the grouped_cuts to a JSON file
    with open(json_file_path, 'w') as json_file:
        json.dump(packed_bars, json_file, indent=4)
    
    bar_counter=0
    for profile, groups_ in packed_bars.items():
        for group in groups_:
            bar_to_use = group['bar_length']
            for cut_ in group['cuts']:
                profile = cut_[0]
                length = cut_[1]
                posNumber = cut_[2]

                cut_info, bar_id = find_best_stock_option(
                    profile, length, posNumber, bars, bar_to_use)
                if cut_info:
                    # Assign bar_id if new bar is created
                    if cut_info['new_bar']:
                        bar_counter += 1

                    results.append({
                        'posNumber': posNumber,
                        'Profile': profile,
                        'Cut Length': length,
                        'Cut Info': cut_info['description'],
                        'Waste After Cut': cut_info['remaining_waste'],
                        'Stock profile POS': bar_id
                    })
                else:
                    print(f"No suitable stock length for cut {length}mm of profile {profile}")

    return results, bars



def find_best_stock_option(profile, length, posNumber, bars, bar_to_use):
    """Determine the best stock bar to cut from, considering existing waste and tracking individual bars."""
   
    # Try to find a waste piece from existing bars
    for bar in bars:
        if bar['profile'] == profile and bar['remaining_waste'] >= length:
            # Cut from existing bar
            bar['cuts'].append({'cut_length': length, 'posNumber': posNumber})
            bar['remaining_waste'] -= length
            return {
                'description': f"Cut from bar {bar['id']} (waste remaining {bar['remaining_waste']}mm)",
                'remaining_waste': bar['remaining_waste'],
                'new_bar': False
            }, bar['id']

    # Create a new bar
    bar_id = f"SN{len(bars) + 1}"
    new_bar = {
        'id': bar_id,
        'profile': profile,
        'stock_length': bar_to_use,
        'cuts': [{'cut_length': length, 'posNumber': posNumber}],
        'remaining_waste': bar_to_use - length
    }
    bars.append(new_bar)
    return {
        'description': f"Cut from new bar {bar_id} ({bar_to_use}mm)",
        'remaining_waste': new_bar['remaining_waste'],
        'new_bar': True
    }, bar_id


In [47]:
projectName = "זהו טקסט בעברית שיומר ל"
results, bars = calculate_cuts(df, stock)
pdfBuffer =save_results_to_dataframe(results,bars,projectName)

import io

output_filename = "Combined_Output.pdf"
with open(output_filename, 'wb') as f:
    f.write(pdfBuffer.getvalue())  # Write the binary content of the buffer to the file
    print(f"PDF saved successfully as {output_filename}")


True
PDF saved successfully as Combined_Output.pdf


In [ ]:
import random
import json
import pandas as pd


data = open('D:\\Python\\python-works\\grouping-minimze-cost-waste\\openai_response.json')
data = json.load(data)
df = pd.json_normalize(data)

# FIND UNIQUE PROFILES
unique_profiles = df['profile'].unique()
print(unique_profiles)

stocks_lengths = {}

for profile in unique_profiles:
    # GENERATE RANDOM NUMBERS 2 - 6 FOR EACH PROFILE

    for i in range(random.randint(2, 6)):
        # GENERATE RANDOM NUMBERS 2 - 6 FOR EACH PROFILE
        random_number = random.randint(2, 6)
        stocks_lengths[profile] = stocks_lengths.get(profile, []) + [random_number]


max_stocks_length = {}
for profile, lengths in stocks_lengths.items():
    max_stocks_length[profile] = max(lengths)
    print(profile, lengths)


# GROUP BY PROFILE
grouped_df = df.groupby('profile')

# NOW LOOP THROUGH EACH PROFILES LENGTH by Splitting
# IT by " " and then take 0 Index and convert it to float
profiles_max_length = {}
for profile, group in grouped_df:
    profiles_max_length[profile] = group['length'].str.split(' ').str[0].astype(float).max()


anomalies = None
for profile, max_length in profiles_max_length.items():

    if max_stocks_length[profile]*1000 < max_length:

        if anomalies is None:
            anomalies = []

        anomalies.append({
            "profile": profile,
            "length": max_length/1000,
        })

for anomaly in anomalies:
    print(anomaly)

['RHS80*3.6' 'RHS60*5' 'RHS120*80*5' 'IPE240' 'HEA220' 'UPN300' 'RHS120*5'
 'HEA260' 'RHS60*120*3.6' 'RHS250*150*6.3' 'RHS40*3.2' 'L70*7' 'RHS50*5'
 'RHS150*6.3' 'RHS200*100*6.3' 'RHS160*80*5' 'RHS80*120*5' 'RHS100*5'
 'NUT_M24']
RHS80*3.6 [6, 6, 3, 3, 2, 6]
RHS60*5 [6, 4, 3]
RHS120*80*5 [3, 5, 3, 2, 3]
IPE240 [5, 3, 3, 4]
HEA220 [3, 2, 5, 6]
UPN300 [4, 3, 6, 5, 2]
RHS120*5 [3, 5, 6, 4]
HEA260 [4, 6]
RHS60*120*3.6 [3, 4, 5]
RHS250*150*6.3 [5, 5, 4, 5]
RHS40*3.2 [5, 3, 2, 6, 4, 4]
L70*7 [4, 6, 3, 3]
RHS50*5 [6, 6, 4, 3, 5, 3]
RHS150*6.3 [2, 5]
RHS200*100*6.3 [2, 4, 3, 3, 5, 6]
RHS160*80*5 [3, 5, 6, 2]
RHS80*120*5 [4, 6, 6, 3]
RHS100*5 [3, 4, 5, 2, 6, 2]
NUT_M24 [6, 2, 4, 2, 4, 4]
{'profile': 'RHS120*80*5', 'length': np.float64(5710.0), 'must_be_greater_or_equal': 5000}
{'profile': 'RHS150*6.3', 'length': np.float64(5390.0), 'must_be_greater_or_equal': 5000}
{'profile': 'RHS250*150*6.3', 'length': np.float64(5386.0), 'must_be_greater_or_equal': 5000}
{'profile': 'RHS60*120*3.6', 'length'